
![texto alternativo](https://raw.githubusercontent.com/Chilangdon20/PYTHON/master/Curso%20Basico/CursoExpress/Imagenes/d.png)

# Transformación de datos usando ``.groupby()``

## Introducción:

En esta sección final , nos centraremos en la familia de funciones  ``groupby``de pandas,nos ayudarán a agrupar las entradas de un DataFrame deacuerdo con los valores de una caracteristica especifica.Para refrescar nuestra memoria, revisaremos cuándo y cómo usar ``.groupby()``.



El dataset que usaremos en este capítulo es una colección de personas que cenan en un restaurante.Para cada persona , tenemos varias características, incluida la cantidad total pagada, la propina dejada al mesero , el dia de la semana y la hora del día.



In [21]:
import pandas as pd
import time
data = pd.read_csv("/content/restaurant_data.csv")
data.head()

,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,Dinner,2
1,10.34,1.66,Male,No,Sun,Dinner,3
2,21.01,3.50,Male,No,Sun,Dinner,3
3,23.68,3.31,Male,No,Sun,Dinner,2
4,24.59,3.61,Female,No,Sun,Dinner,4


### ¿Fumadores VS NoFumadores?

El método ``groupby()``se aplica a un DataFrame y lo agrupa según una caracteristica.Luego ,podemos aplicar algunas funciónes simples o más complicadas en ese objeto agrupado.

In [3]:
grupo_fumadores = data.groupby('smoker')

El método más simple para llamar es ``count``,al,principio agrupamos los datos del restaurante según si el cliente era fumador o no.

Ahora aplicaremos el metodo ``count`` para obtener el recuento de fumadores y no fumadores.No es sorprendente que obtengamos los mismos resultados para todas las funciones.El metodo ``count``cuenta el número de ocurrencias de cada grupo en cada característica.

Como nuestros datos no tiene valores faltantes, los resultados deben ser los mismo en todas las columnas.

In [4]:
print(grupo_fumadores.count())

        total_bill  tip  sex  day  time  size
smoker                                       
No             151  151  151  151   151   151
Yes             93   93   93   93    93    93


## Transformación de Datos.

Despues de agrupar las entradas del DataFrame deacuerdo con los valores de una caracteristica especifica , podemos aplicar cualquier tipo de transformación que nos interesa.


Aqui aplicaremos *z-score* , una transformación de normalización, que es la distancia entre cada valor y la **media**, dividida por la desviación estándar.


$$Z Score =\frac{ (x – μ)}{ơ}$$

Esta es un transformacion muy útil en estadística , a menudo utilizada en pruebas estandarizadas.

In [15]:
zscore = lambda x: (x - x.mean() ) / x.std()

Para aplicar esta transformación al objeto agrupado , solo necesitamo llamar el metodo ``transform``que contiene la transformación lambda que definimos.

In [22]:
start_time = time.time()
rest_agrupado = data.groupby('size')
rest_transformacion = rest_agrupado.transform(zscore)
print("El tiempo usando el metodo groupby es : {} segundos".format(time.time()-start_time))

El tiempo usando el metodo groupby es : 0.02555394172668457 segundos


Para cada elemento , restamos la media y la dividimos por la desviación estándar del grup al que pertenece.Tambien podemos ver que se aplican solo a las característica numéricas del DataFrame.



Si bien el método ``transform()``simplifica mucholas cosas...
¿Es realmente más eficiente que usar el código nativo de Python?

Como lo hicimos antes:

* Agrupamos nuestros datos por sexo & aplicamos la funcion *zscore*


In [ ]:
start_time = time.time()
data.groupby('sex').transform(zscore)

mean_f = data.groupby('sex').mean()['total_bill']['Female']
mean_m = data.groupby('sex').mean()['total_bill']['Male']
std_female = data.groupby('sex').std()['total_bill']['Female']
std_male = data.groupby('sex').std()['total_bill']['Male']

for i in range(len(data)):
  if data.iloc[i][2] == 'Female':
    data.iloc[i][0] = (data.iloc[i][0] - mean_f)/std_female
  else:
    data.iloc[i][0] = (data.iloc[i][0] - mean_m)/std_male



In [25]:
print("Tiempo usando codigo nativo : {} segundos ".format(time.time()-start_time))

Tiempo usando codigo nativo : 3.092251777648926 segundos 


Podemos ver que con el uso de lafunción ``trasform``logramos una mejora masiva de la velocidad , ademas de eso , solo usamos una linea para realizar la operacion de interes.

## Filtrado de datos usando ``filter``.

En esta ultima seccion discutiremos cómpo podemos usar la función ``filter()``en un objeto e pandas agrupado, esto nos permite incluir solo un subconjunto de esos grupos , en función de algunas condiciones especificas.

A menudo , después de agrupar las entradas de un DataFrame deacuerdo con una característica específica estamos interesados en inclluir solo un subconjunto de esos grupos en función de algunas condiciones.Algunos valores especificos son :

* Numero de valores faltantes.

* Media de alguna caracteristica en especifico.

* Numero de ocurrencias en un DataSet.

Estamos interesados en encontrar la cantidad media de salarios dados en los que el monto promedio pagado al mesero es más de 20 USD.


La función ``filter``acepta una función lambda que oper en un DataFrame de cada uno de los grupos.

En este ejemplo la funcion *lambda* selecciona la columna *total_bill* y comprueba que la media es mayor que 20, si esa función lambda devuelve True , se calcula la media.


In [33]:
s_t = time.time()
rest_grouped = data.groupby('day')
filt_trans= lambda x: x['total_bill'].mean() > 20
rest_filt = rest_grouped.filter(filt_trans)
print("Tiempo usando ..filter(): {} segundos".format(time.time()-s_t))

Tiempo usando ..filter(): 0.0067141056060791016 segundos


Si comparamos con la media total de las propinas podemos ver que hay una diferencia entre los dos valores, lo que significa que el filtrado se realizó correctamente.

In [35]:
print(rest_filt['tip'].mean())

3.1152760736196328


In [36]:
print(data['tip'].mean())

2.9982786885245902
